In [2]:
import pandas as pd
import numpy as np
import os

data=pd.read_csv(str(os.getcwd())+'/tennis.csv')

In [3]:
data.set_index('day',inplace=True)

In [4]:
data

,outlook,temp,humidity,wind,play
day,,,,,
D1,Sunny,Hot,High,Weak,No
D2,Sunny,Hot,High,Strong,No
D3,Overcast,Hot,High,Weak,Yes
D4,Rain,Mild,High,Weak,Yes
D5,Rain,Cool,Normal,Weak,Yes
D6,Rain,Cool,Normal,Strong,No
D7,Overcast,Cool,Normal,Strong,Yes
D8,Sunny,Mild,High,Weak,No
D9,Sunny,Cool,Normal,Weak,Yes


In [5]:
class Node:
    def __init__(self,value,nxt_node=None,prev_node=None):
        self.nxtnode=nxt_node
        self.value=value
        self.prevnode=prev_node
        self.leaf_node=False
    
    def get_value(self):
        return self.value

    def set_prevnode(self,prevnode):
        self.prevnode=prevnode

    def get_prevnode(self):
        return self.prevnode

    def set_nxtnode(self,n_node):
        self.nxtnode=n_node

    def get_nxtnode(self):
        return self.nxtnode

In [6]:
#implementing the ID3 algorithm.
class ID3:
    def __init__(self,dataset):
        self.data=dataset
        self.total=dataset.shape[0]
        #assuming target variable would be at the end of the list.
        self.target_var=list(data.columns)[-1]
        
    def entropy(self,feature_data=None):
        #entropy is a measure of the distortion or the amount of diversity
        #in terms of labels or target variable present in the dataset.
        #an entropy value of 0 means only single category is present whereas an
        #entropy value of 1 means the dataset is perfectly balanced and all categories 
        #have equal number of rows.
        if feature_data is not None:
            data=feature_data
            total=data.shape[0]
        else:
            data=self.data
            total=self.total
        self.n=data[self.target_var].nunique()
        self.labels=list(data[self.target_var].unique())
        self.prob=[]
        self.ent=0
        
        for i in range(1,self.n+1):
            prob=(data[data[self.target_var]==self.labels[i-1]].shape[0])/total
            self.prob.append(prob)
            ent=-prob*np.log2(prob)
            #print(ent)
            self.ent=self.ent+ent
        return self.ent
    
    def entropy_per_feature(self,fname,feature_data=None):
        if feature_data is not None:
            data=feature_data
        else:
            data=self.data
        unique_values=list(data[fname].unique())
        #print(unique_values)
        unique_val_count=data[fname].nunique()
        ent_ftr=0
        for i in range(0,unique_val_count):
            category_data=data[data[fname]==unique_values[i]]
            #print(type(category_data))
            ent_cat=(category_data.shape[0]/self.total)*self.entropy(category_data)
            #print(ent_cat)
            ent_ftr=ent_ftr+ent_cat
        return ent_ftr
    
    def information_gain(self,fname,feature_data=None): 
        #print(fname) 
        if feature_data is None:
            information_gain=self.entropy()-self.entropy_per_feature(fname)
        else:
            et=self.entropy(feature_data)
            if et==0:
                return 0
            information_gain=et-self.entropy_per_feature(fname,feature_data)
        return information_gain

In [7]:
#maximum value function.
def max_val(dct):
    max_val=0
    key_max=''
    for key in dct.keys():
        if dct[key]>max_val:
            max_val=dct[key]
            key_max=key
    return key_max

In [8]:
def oop_building_tree():
    algo=ID3(data)
    feature_list=list(data.columns)[:-1]
    ig={}
    root_node=''
    #finding the root node.
    for feature in feature_list:
        i_gain=algo.information_gain(feature)
        ig[feature]=i_gain
    root_node=max_val(ig)
    del ig[root_node]
    n=Node(root_node)
    print("root_node:{}".format(n.get_value()))
    #finding child nodes of the root node.
    category_list=list(data[n.get_value()].unique())
    print("{}=>{}".format(n.get_value(),category_list))
    n_list=[Node(category) for category in category_list]
    #establishing links between the child nodes and the root node(parent node).
    n.set_nxtnode(n_list)
    for node in n_list:
        node.set_prevnode(n)
    #finding remaining intermediate nodes or decision nodes.
    while (len(ig.keys())!=0 and len(feature_list)!=1):
        for node in n_list:
            cat_data=data[data[n.get_value()]==node.get_value()]
            ig_1={}
            count=0
            for key in ig.keys():
                i_gain=algo.information_gain(key,cat_data)
                if i_gain==0:
                    #print(node.get_value())
                    values=list(cat_data.play.unique())
                    #values=['Yes']
                    val_node=[Node(val) for val in values]
                    node.set_nxtnode(val_node)
                    for vnode in val_node:
                        vnode.set_prevnode(node)
                        #print(vnode.get_nxtnode())
                        vnode.leaf_node=True
                    count=1
                    #node.leaf_node=True
                    print("{}=>{}".format(node.get_value(),list(data.play[data[n.get_value()]==node.get_value()].unique())[0]))
                    del ig[key]
                    break  
                ig_1[key]=i_gain
            if count==1:
                #print("star")
                continue
            root1_node=max_val(ig_1)
            #establishing links between the newly found intermediate node.
            #and the respectie parent node.
            intr_node=Node(root1_node)
            node.set_nxtnode(intr_node)
            intr_node.set_prevnode(node)
            print("{}=>{}".format(node.get_value(),intr_node.get_value()))
            intr_values=[Node(val) for val in list(data[intr_node.get_value()].unique())]
            intr_node.set_nxtnode(intr_values)
            print("{}=>{}".format(intr_node.get_value(),[node.get_value() for node in intr_node.get_nxtnode()]))
            for vnode in intr_values:
                vnode.set_prevnode(intr_node)
                #print(vnode.get_nxtnode())
                vnode.leaf_node=True
            #print(root1_node)
            del ig[str(root1_node)]
                
        
        


In [9]:
oop_building_tree()

root_node:outlook
outlook=>['Sunny', 'Overcast', 'Rain']
Sunny=>humidity
humidity=>['High', 'Normal']
Overcast=>Yes
Rain=>wind
wind=>['Weak', 'Strong']
